In [ ]:
import subprocess
import neutralb1.utils as utils

WORKSPACE_DIR = utils.get_workspace_dir()

git_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD'], cwd=WORKSPACE_DIR).decode('utf-8').strip()
print(git_hash)

# Comparing PWA Results With New Event Selection
We've updated our event selection using FSRoot and a handful of new cuts, reducing our statistics, and potentially affecting the kinematics of our events such that the PWA could substantially change. Here we will compare the PWA results using this new event selection to our results obtained from our old event selection that has much looser cuts. The same model will be used:
* Only spin-1 ($J=1$) waves
  * {$J^P\ell$} = {$1^+S, 1^-P, 1^+D$}
* The D and S waves are constrained to each other, such that every $D$-wave has a common ratio and phase between its $S$-wave partner
  * Boundaries: $0< r_{D/S} < 1,\quad -\pi<\phi_{D-S}<\pi$
* Include an isotropic background term
  * Assumed that sideband subtraction handles almost all background events, and so these events will be *true* $\omega$ events, but contribute in total a flat distribution to the angular distributions
* Uses all 4 polarization orientations

The previous notebook (no output) [can be found here](./constrained_ratio.ipynb), and the presentation + complete notebook is [available online](https://halldweb.jlab.org/doc-private/DocDB/ShowDocument?docid=7027). A crucial difference in the results between these two is that the previous study utilized 100 bootstrap fits per bin to better approximate the errors on the parameters. To keep this comparison study fast, we do no bootstrapping, and so all errors are MINUIT only. Previous studies have shown our MINUIT errors tend to be roughly 1/2 the size of the bootstrap errors.

The details of the fit can be described as:
* Split data into bins of mass and $-t$
  * 20 MeV wide bins from $1.0 < M_{\omega\pi^0} < 2.0$ 
  * Focus only on the $0.1 < -t < 0.2$ range for now
* Perform 500 randomized fits in each bin of mass and $-t$ independently 

## Setup


In [ ]:
# load common libraries
import pandas as pd
import pickle as pkl
import pathlib
import os, sys
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict

# load neutralb1 libraries
import neutralb1.utils as utils
from neutralb1.analysis.result import ResultManager

utils.load_environment()

# load in useful directories as constants
CWD = pathlib.Path.cwd()
STUDY_DIR = f"{WORKSPACE_DIR}/studies/data-fits/fsroot-check/"

# set env variables for shell cells
os.environ["WORKSPACE_DIR"] = WORKSPACE_DIR
os.environ['STUDY_DIR'] = STUDY_DIR

In [ ]:
%%bash
# print out yaml file used to submit the fits
cat $STUDY_DIR/submission.YAML

In [ ]:
# load in the preprocessed results 
with open(f"{STUDY_DIR}/t_0.1-0.2/preprocessed_results.pkl", "rb") as f:
    data = pkl.load(f)
    new_results = ResultManager(**data)
with open(f"{STUDY_DIR}/t_0.1-0.2/preprocessed_results_acceptance_corrected.pkl", "rb") as f:
    data = pkl.load(f)
    new_results_ac = ResultManager(**data)

with open(f"{STUDY_DIR}/../spin-1-constrained-ratio/t_0.1-0.2/preprocessed_results.pkl", "rb") as f:
    data = pkl.load(f)
    old_results = ResultManager(**data)
with open(f"{STUDY_DIR}/../spin-1-constrained-ratio/t_0.1-0.2/preprocessed_results_acceptance_corrected.pkl", "rb") as f:
    data = pkl.load(f)
    old_results_ac = ResultManager(**data)


In [ ]:
new_results.summary()

In [ ]:
old_results.summary()

## Analysis

### Standard Plots
First step is to print out all the standard plots to view the model's performance

In [ ]:
utils.big_print(f"t bin: 0.1-0.2", 2.0)
for pdf in pathlib.Path(f"{STUDY_DIR}/t_0.1-0.2/plots").glob("*.pdf"):
    if pdf.name in ["jp.pdf", "waves.pdf", "matrix.pdf", "proj_moments.pdf"]:
        utils.display_pdf(str(pdf), 0)

In [ ]:
utils.big_print(f"t bin: 0.1-0.2", 2.0)
for pdf in pathlib.Path(f"{STUDY_DIR}/../spin-1-constrained-ratio/t_0.1-0.2/plots").glob("*.pdf"):
    if pdf.name in ["jp.pdf", "waves.pdf", "matrix.pdf", "proj_moments.pdf"]:
        utils.display_pdf(str(pdf), 0)

### D/S Ratio
Compare the D/S Ratio to E852 results, and check for correlations using the bootstrap fits

In [ ]:
plt.rcParams["figure.dpi"] = 300
new_results.plot.diagnostic.ds_ratio()

In [ ]:
# The result class does not yet have the correlation matrix loaded, but we do have the dataframe to access it
corr_df = pd.read_csv(f"{STUDY_DIR}/t_0.1-0.2/raw/best_corr.csv")
ratio_corrs = corr_df[corr_df["parameter"] == "dsratio"]["dphase"]

In [ ]:
mass = new_results.plot.intensity._masses
fig, ax = plt.subplots(figsize=(12,3))
plt.plot(mass, ratio_corrs, marker=".", linestyle="none", color="black")
plt.xlabel(rf"$\omega\pi^0$ inv. mass $(GeV)$", loc="right")
plt.ylabel(r"$\rho(D/S, D-S)$", loc="top")
plt.ylim(-1, 1)

In [ ]:
plt.rcParams["figure.dpi"] = 300
old_results.plot.diagnostic.ds_ratio()

### b1 - Vector Interaction

In [ ]:
new_results.plot.phase.mass_phase("p1p0S", "p1mpP", {"color": "tab:blue"}, {"color": "tab:orange"})

In [ ]:
old_results.plot.phase.mass_phase("p1p0S", "p1mpP", {"color": "tab:blue"}, {"color": "tab:orange"})

### Acceptance-Corrected Coherent Sums

In [ ]:
fig, axs = plt.subplots(
    2,
    1,    
    sharey="row",
    sharex=True,
    figsize=(8, 7),
    layout="constrained",
    squeeze=False
)

# Store handles and labels for the legend
handles = []
labels = []
bin_width = None


masses = new_results_ac.plot.intensity._masses
bin_width = new_results_ac.plot.intensity._bin_width

# ##### 1P COHERENT SUM #####  

h1 = axs[0, 0].errorbar(
    masses, 
    new_results_ac.fit_df["p1p"],
    new_results_ac.fit_df["p1p_err"],
    bin_width / 2,
    marker="",
    linestyle="-",
    markersize=6,
    color="darkred",
    label=rf"$1^+$ (Natural)",
)
h2 = axs[0, 0].errorbar(
    masses, 
    new_results_ac.fit_df["p1m"],
    new_results_ac.fit_df["p1m_err"],
    bin_width / 2,
    marker="",
    linestyle="-",
    markersize=6,
    color="lightcoral",
    label=rf"$1^-$ (Natural)",
)

# ##### 1M COHERENT SUM #####

h3 = axs[1, 0].errorbar(
    masses, 
    new_results_ac.fit_df["m1p"],
    new_results_ac.fit_df["m1p_err"],
    bin_width / 2,
    marker="",
    linestyle="-",
    markersize=6,
    color="darkblue",
    label=rf"$1^+$ (Unnatural)",
)
h4 = axs[1, 0].errorbar(
    masses, 
    new_results_ac.fit_df["m1m"],
    new_results_ac.fit_df["m1m_err"],
    bin_width / 2,
    marker="",
    linestyle="-",
    markersize=6,
    color="lightblue",
    label=rf"$1^-$ (Unnatural)",
)

# Collect legend handles and labels only from the first subplot to avoid duplicates
handles.extend([h1, h2, h3, h4])
labels.extend([h.get_label() for h in [h1, h2, h3, h4]])

axs[0, 0].set_title(rf" 0.1 < $-t$ < 0.2 GeV$^2$", pad=10, fontsize=16)
axs[0, 0].set_ylim(bottom=0.0)
axs[1, 0].set_ylim(bottom=0.0)
axs[1, 0].set_xlabel(rf"$\omega\pi^0$ inv. mass $(GeV)$", loc="right")

axs[0, 0].set_ylabel(f"Events / {bin_width:.3f} GeV", loc="top")
axs[1, 0].set_ylabel(f"Events / {bin_width:.3f} GeV", loc="top")
axs[1, 0].ticklabel_format(axis='y', style='sci', scilimits=(0,0))

# Create a single legend for the entire figure positioned on the right
fig.legend(handles, labels, loc='center right', bbox_to_anchor=(1.33, 0.5))

In [ ]:
fig, axs = plt.subplots(
    2,
    1,    
    sharey="row",
    sharex=True,
    figsize=(8, 7),
    layout="constrained",
    squeeze=False
)

# Store handles and labels for the legend
handles = []
labels = []
bin_width = None


masses = old_results_ac.plot.intensity._masses
bin_width = old_results_ac.plot.intensity._bin_width

# ##### 1P COHERENT SUM #####  

h1 = axs[0, 0].errorbar(
    masses, 
    old_results_ac.fit_df["p1p"],
    old_results_ac.plot.intensity.get_bootstrap_error("p1p"),
    bin_width / 2,
    marker="",
    linestyle="-",
    markersize=6,
    color="darkred",
    label=rf"$1^+$ (Natural)",
)
h2 = axs[0, 0].errorbar(
    masses, 
    old_results_ac.fit_df["p1m"],
    old_results_ac.plot.intensity.get_bootstrap_error("p1m"),
    bin_width / 2,
    marker="",
    linestyle="-",
    markersize=6,
    color="lightcoral",
    label=rf"$1^-$ (Natural)",
)

# ##### 1M COHERENT SUM #####

h3 = axs[1, 0].errorbar(
    masses, 
    old_results_ac.fit_df["m1p"],
    old_results_ac.plot.intensity.get_bootstrap_error("m1p"),
    bin_width / 2,
    marker="",
    linestyle="-",
    markersize=6,
    color="darkblue",
    label=rf"$1^+$ (Unnatural)",
)
h4 = axs[1, 0].errorbar(
    masses, 
    old_results_ac.fit_df["m1m"],
    old_results_ac.plot.intensity.get_bootstrap_error("m1m"),
    bin_width / 2,
    marker="",
    linestyle="-",
    markersize=6,
    color="lightblue",
    label=rf"$1^-$ (Unnatural)",
)

# Collect legend handles and labels only from the first subplot to avoid duplicates
handles.extend([h1, h2, h3, h4])
labels.extend([h.get_label() for h in [h1, h2, h3, h4]])

axs[0, 0].set_title(rf" 0.1 < $-t$ < 0.2 GeV$^2$", pad=10, fontsize=16)
axs[0, 0].set_ylim(bottom=0.0)
axs[1, 0].set_ylim(bottom=0.0)
axs[1, 0].set_xlabel(rf"$\omega\pi^0$ inv. mass $(GeV)$", loc="right")

axs[0, 0].set_ylabel(f"Events / {bin_width:.3f} GeV", loc="top")
axs[1, 0].set_ylabel(f"Events / {bin_width:.3f} GeV", loc="top")
axs[1, 0].ticklabel_format(axis='y', style='sci', scilimits=(0,0))

# Create a single legend for the entire figure positioned on the right
fig.legend(handles, labels, loc='center right', bbox_to_anchor=(1.33, 0.5))

### Fit Fraction Comparison
Since our statistics have changed with the new event selection, we can only directly compare the results by their fit fraction (wave intensity / total intensity). Let's compare the coherent sums and wave fit fractions here

#### Individual Waves

In [ ]:
from uncertainties import ufloat, unumpy

for wave in new_results.coherent_sums["eJPmL"]:
    mass = new_results.plot.intensity._masses
    new_total = unumpy.uarray(new_results.fit_df["detected_events"].to_numpy(), new_results.fit_df["detected_events_err"].to_numpy())
    old_total = unumpy.uarray(old_results.fit_df["detected_events"].to_numpy(), old_results.fit_df["detected_events_err"].to_numpy())
    new_wave = unumpy.uarray(new_results.fit_df[wave].to_numpy(), new_results.fit_df[f"{wave}_err"].to_numpy())
    old_wave = unumpy.uarray(old_results.fit_df[wave].to_numpy(), old_results.fit_df[f"{wave}_err"].to_numpy())
    new_frac = new_wave / new_total
    old_frac = old_wave / old_total

    resid = new_frac - old_frac    

    fig, (ax_top, ax_bot) = plt.subplots(
        nrows=2,
        sharex=True,
        figsize=(8, 6),
        dpi=100,
        gridspec_kw={"height_ratios": [3, 1], "hspace": 0.05},
    )

    ax_top.errorbar(
        mass,
        unumpy.nominal_values(new_frac),
        yerr=unumpy.std_devs(new_frac),
        fmt="o",
        label="New Fit",
        color="tab:blue",
    )
    ax_top.errorbar(
        mass,
        unumpy.nominal_values(old_frac),
        yerr=unumpy.std_devs(old_frac),
        fmt="o",
        markerfacecolor="none",
        markeredgecolor="tab:orange",
        color="tab:orange",
        label="Old Fit",
    )
    ax_top.set_ylabel(rf"Fit Fraction", loc="top")
    ax_top.set_title(utils.convert_amp_name(wave), fontsize=22)
    ax_top.legend()

    ax_bot.errorbar(
        mass,
        unumpy.nominal_values(resid),
        yerr=unumpy.std_devs(resid),
        fmt=".",
        color="black",
    )
    bound = max(
        abs((unumpy.nominal_values(resid)+unumpy.std_devs(resid)).max()), 
        abs((unumpy.nominal_values(resid)-unumpy.std_devs(resid)).min()))
    ax_bot.set_ylim(-(bound+0.02*bound) , bound+0.02*bound)
    ax_bot.axhline(0, color="gray", linestyle="--", linewidth=1)
    ax_bot.set_xlabel(rf"$\omega\pi^0$ inv. mass $(GeV)$", loc="right")
    ax_bot.set_ylabel("Δ", rotation=90, labelpad=10, loc="center")

    plt.show()

#### $1^+S_0^{(+)} - 1^-P_1^{(+)}$

In [ ]:
# plot the s - P phase difference comparison too
mass = new_results.plot.intensity._masses
fig, (ax_top, ax_bot) = plt.subplots(
    nrows=2,
    sharex=True,
    figsize=(8, 6),
    gridspec_kw={"height_ratios": [3, 1], "hspace": 0.05},
)
phase = new_results.phase_difference_dict[("p1p0S", "p1mpP")]
new_phase = unumpy.uarray(new_results.fit_df[phase].to_numpy(), new_results.fit_df[f"{phase}_err"].to_numpy())
old_phase = unumpy.uarray(old_results.fit_df[phase].to_numpy(), old_results.fit_df[f"{phase}_err"].to_numpy())
vectorized_circ_res = np.vectorize(utils.circular_residual)
resid = vectorized_circ_res(
    np.abs(unumpy.nominal_values(new_phase)), 
    np.abs(unumpy.nominal_values(old_phase)),
    in_degrees=True
)
resid = unumpy.uarray(resid, unumpy.std_devs(new_phase - old_phase)) 

ax_top.errorbar(
    mass,
    unumpy.nominal_values(new_phase),
    yerr=unumpy.std_devs(new_phase),
    fmt="o",
    label="New Fit",
    color="tab:blue",
)
ax_top.errorbar(
    mass,
    -unumpy.nominal_values(new_phase),
    yerr=unumpy.std_devs(new_phase),
    fmt="o",    
    color="tab:blue",
)

ax_top.errorbar(
    mass,
    unumpy.nominal_values(old_phase),
    yerr=unumpy.std_devs(old_phase),
    fmt="o",
    markerfacecolor="none",
    markeredgecolor="tab:orange",
    color="tab:orange",
    label="Old Fit",
)
ax_top.errorbar(
    mass,
    -unumpy.nominal_values(old_phase),
    yerr=unumpy.std_devs(old_phase),
    fmt="o",
    markerfacecolor="none",
    markeredgecolor="tab:orange",
    color="tab:orange",    
)

ax_top.set_ylabel(rf"Phase Difference", loc="top")
ax_top.set_title(utils.convert_amp_name(phase), fontsize=22)
ax_top.legend()

ax_bot.errorbar(
    mass,
    unumpy.nominal_values(resid),
    yerr=unumpy.std_devs(resid),
    fmt=".",
    color="black",
)
bound = max(
    abs((unumpy.nominal_values(resid)+unumpy.std_devs(resid)).max()), 
    abs((unumpy.nominal_values(resid)-unumpy.std_devs(resid)).min()))
ax_bot.set_ylim(-(bound+0.02*bound) , bound+0.02*bound)
ax_bot.axhline(0, color="gray", linestyle="--", linewidth=1)
ax_bot.set_xlabel(rf"$\omega\pi^0$ inv. mass $(GeV)$", loc="right")
ax_bot.set_ylabel("Δ", rotation=90, labelpad=10, loc="center")
plt.show()

#### Acceptance-Corrected Coherent Sums

In [ ]:
from uncertainties import ufloat, unumpy

mass = new_results_ac.plot.intensity._masses

# Calculate fit fractions for all coherent sums
new_total = unumpy.uarray(
    new_results_ac.fit_df["generated_events"].to_numpy(), 
    new_results_ac.fit_df["generated_events_err"].to_numpy()
)
old_total = unumpy.uarray(
    old_results_ac.fit_df["generated_events"].to_numpy(), 
    old_results_ac.fit_df["generated_events_err"].to_numpy()
)

# Get coherent sums and errors
coherent_sums = {
    "p1p": "Natural $1^+$",
    "p1m": "Natural $1^-$", 
    "m1p": "Unnatural $1^+$",
    "m1m": "Unnatural $1^-$"
}

fractions = {}
for wave in coherent_sums.keys():
    new_wave = unumpy.uarray(
        new_results_ac.fit_df[wave].to_numpy(), 
        new_results_ac.fit_df[f"{wave}_err"].to_numpy()
    )
    old_wave = unumpy.uarray(
        old_results_ac.fit_df[wave].to_numpy(),
        old_results_ac.plot.intensity.get_bootstrap_error(wave)
    )
    
    fractions[wave] = {
        "new": new_wave / new_total,
        "old": old_wave / old_total,
        "resid": (new_wave / new_total) - (old_wave / old_total)
    }

# Create 2x2 subplot grid
fig, axs = plt.subplots(
    2, 2,
    figsize=(16, 10),
    gridspec_kw={"height_ratios": [3, 1], "hspace": 0.08, "wspace": 0.25}
)

# Natural parity (left column)
natural_waves = ["p1p", "p1m"]
for wave, label in [(w, coherent_sums[w]) for w in natural_waves]:
    color = "darkred" if wave == "p1p" else "lightcoral"
    marker= "o" if wave == "p1p" else "s"
    axs[0, 0].errorbar(
        mass,
        unumpy.nominal_values(fractions[wave]["new"]),
        yerr=unumpy.std_devs(fractions[wave]["new"]),
        marker=marker,
        color=color,
        linestyle="",
        label=f"{label} (New)",
    )
    axs[0, 0].errorbar(
        mass,
        unumpy.nominal_values(fractions[wave]["old"]),
        yerr=unumpy.std_devs(fractions[wave]["old"]),
        marker=marker,
        markerfacecolor="none",
        markeredgecolor=color,
        color=color,
        linestyle="",
        label=f"{label} (Old)",
    )

axs[0, 0].set_ylabel("Fit Fraction", loc="top")
axs[0, 0].set_title("Natural Parity", fontsize=16)
axs[0, 0].legend(loc="best")

# Natural parity residuals
for wave in natural_waves:
    color = "darkred" if wave == "p1p" else "lightcoral"
    marker = "o" if wave == "p1p" else "s"
    resid = fractions[wave]["resid"]
    axs[1, 0].errorbar(
        mass,
        unumpy.nominal_values(resid),
        yerr=unumpy.std_devs(resid),
        marker=marker,
        markersize=4,
        color=color,
        linestyle="",
        label=coherent_sums[wave]
    )

bound = max([
    abs((unumpy.nominal_values(fractions[w]["resid"]) + unumpy.std_devs(fractions[w]["resid"])).max())
    for w in natural_waves
] + [
    abs((unumpy.nominal_values(fractions[w]["resid"]) - unumpy.std_devs(fractions[w]["resid"])).min())
    for w in natural_waves
])
axs[1, 0].set_ylim(-(bound + 0.02 * bound), bound + 0.02 * bound)
axs[1, 0].axhline(0, color="gray", linestyle="--", linewidth=1)
axs[1, 0].set_xlabel(rf"$\omega\pi^0$ inv. mass $(GeV)$", loc="right")
axs[1, 0].set_ylabel("Δ", rotation=90, labelpad=10, loc="center")

# Unnatural parity (right column)
unnatural_waves = ["m1p", "m1m"]
for wave, label in [(w, coherent_sums[w]) for w in unnatural_waves]:
    color = "darkblue" if wave == "m1p" else "lightblue"
    marker = "o" if wave == "m1p" else "s"
    axs[0, 1].errorbar(
        mass,
        unumpy.nominal_values(fractions[wave]["new"]),
        yerr=unumpy.std_devs(fractions[wave]["new"]),
        marker=marker,
        color=color,
        linestyle="",
        label=f"{label} (New)",
    )
    axs[0, 1].errorbar(
        mass,
        unumpy.nominal_values(fractions[wave]["old"]),
        yerr=unumpy.std_devs(fractions[wave]["old"]),
        marker=marker,
        markerfacecolor="none",
        markeredgecolor=color,
        color=color,
        linestyle="",
        label=f"{label} (Old)",
    )

axs[0, 1].set_ylabel("Fit Fraction", loc="top")
axs[0, 1].set_title("Unnatural Parity", fontsize=16)
axs[0, 1].legend(loc="best")

# Unnatural parity residuals
for wave in unnatural_waves:
    color = "darkblue" if wave == "m1p" else "lightblue"
    marker = "o" if wave == "m1p" else "s"
    resid = fractions[wave]["resid"]
    axs[1, 1].errorbar(
        mass,
        unumpy.nominal_values(resid),
        yerr=unumpy.std_devs(resid),
        marker=marker,
        markersize=4,
        color=color,
        linestyle="",
        label=coherent_sums[wave]
    )

bound = max([
    abs((unumpy.nominal_values(fractions[w]["resid"]) + unumpy.std_devs(fractions[w]["resid"])).max())
    for w in unnatural_waves
] + [
    abs((unumpy.nominal_values(fractions[w]["resid"]) - unumpy.std_devs(fractions[w]["resid"])).min())
    for w in unnatural_waves
])
axs[1, 1].set_ylim(-(bound + 0.02 * bound), bound + 0.02 * bound)
axs[1, 1].axhline(0, color="gray", linestyle="--", linewidth=1)
axs[1, 1].set_xlabel(rf"$\omega\pi^0$ inv. mass $(GeV)$", loc="right")
axs[1, 1].set_ylabel("Δ", rotation=90, labelpad=10, loc="center")

plt.suptitle("Acceptance-Corrected Coherent Sum Fit Fraction Comparison", fontsize=18, y=0.995)
plt.show()